In [1]:
#MAT 254 - University of Bergen
#Compare TPFA and MPFA

import numpy as np
import scipy
import sympy

import porepy as pp

# 1. Input and Creation of the grid
Nx = Ny = 4
Lx = Ly = 1
phys_dims = [Lx,Ly]
g = pp.CartGrid([Nx, Ny], phys_dims)
g.compute_geometry()
# Define boundary faces
top_faces = np.ravel(np.argwhere(g.face_centers[1] > Ly - 1e-10))
bot_faces = np.ravel(np.argwhere(g.face_centers[1] < 1e-10))
lef_faces = np.ravel(np.argwhere(g.face_centers[0] < 1e-10))
rig_faces = np.ravel(np.argwhere(g.face_centers[0] > Lx - 1e-10))

pert = 0.3

if pert > 0:
    top_nodes = np.ravel(np.argwhere(g.nodes[1] > Ly - 1e-10))
    bot_nodes = np.ravel(np.argwhere(g.nodes[1] < 1e-10))
    lef_nodes = np.ravel(np.argwhere(g.nodes[0] < 1e-10))
    rig_nodes = np.ravel(np.argwhere(g.nodes[0] > Lx - 1e-10))
    
    move = np.ones(g.num_nodes)
    move[top_nodes] = 0
    move[bot_nodes] = 0
    move[lef_nodes] = 0
    move[rig_nodes] = 0

    
    dx = 1/Nx
    rand = np.vstack((np.random.rand(g.dim, g.num_nodes), np.repeat(0., g.num_nodes)))
    g.nodes += pert * dx * (rand - 0.5)*move
    if g.dim == 2:
        g.nodes[2, :] = 0
    g.compute_geometry()
    

xc = g.cell_centers
xf = g.face_centers
xn = g.nodes
cell_id = np.arange(g.num_cells)
bound_faces = g.get_all_boundary_faces()
pp.plot_grid(g, cell_value=cell_id, info='cf', alpha=0.0, figsize=(8,8))
# Analytical solution
x, y = sympy.symbols('x y')
p = x**2*y**3 #linear pressure should give 0 error
pi = sympy.lambdify((x, y), p, 'numpy')
dpx = sympy.diff(p, x)
dpy = sympy.diff(p, y)
dpxi = sympy.lambdify((x, y), dpx, 'numpy')
dpyi = sympy.lambdify((x, y), dpy, 'numpy')
rhs = -sympy.diff(dpx, x) - sympy.diff(dpy, y)
fi = sympy.lambdify((x, y), rhs, 'numpy')

pc_ex = pi(xc[0], xc[1])
pf_ex = pi(xf[0], xf[1])
flux_ex = -(g.face_normals[0,:] * dpxi(xf[0], xf[1]) + g.face_normals[1,:] * dpyi(xf[0], xf[1]))

# Permeability

perm_xx = 1
perm_yy = 1
perm_xy = 0
k_xx = perm_xx * np.ones(g.num_cells)
k_xy = perm_xy * np.ones(g.num_cells)
k_yy = perm_yy * np.ones(g.num_cells)
k = pp.SecondOrderTensor(k_xx, kyy=k_yy, kxy=k_xy)
# Unitary scalar source already integrated in each cell

fc = fi(xc[0, :], xc[1, :])*g.cell_volumes

#### Boundary conditions - case 2
# Define boundary type: Dirichlet at left and right, Neunam at top and bottom

neuman = []
dirich = np.block([lef_faces, rig_faces, bot_faces, top_faces])

#neuman = np.block([top_faces])
#dirich = np.block([lef_faces, rig_faces, bot_faces, top_faces])


if len(neuman) == 0:
    bound_faces = np.block([dirich])
    labels = np.array(['dir']*bound_faces.size)
    p_bound = np.zeros(g.num_faces)
    p_bound[dirich] = pf_ex[dirich]
else:
    bound_faces = np.block([neuman,dirich])
    labels = np.array(['dir']*bound_faces.size)
    labels[0:len(neuman)] = 'neu' 
    p_bound = np.zeros(g.num_faces)
    p_bound[dirich] = pf_ex[dirich]
    p_bound[neuman] = flux_ex[neuman]
    
bound_condition = pp.BoundaryCondition(g, bound_faces, labels)
# Collect all parameters in a dictionary
parameters = {"second_order_tensor": k, "bc": bound_condition, "bc_values": p_bound}
data_key = "flow"
data = pp.initialize_default_data(g, {}, data_key, parameters)
matrix_dictionary = data[pp.DISCRETIZATION_MATRICES][data_key]
######## TPFA
# Two-point flux approximation - compute pressure
tpfa_solver = pp.Tpfa(data_key)
tpfa_solver.discretize(g, data)
A, b_flow = tpfa_solver.assemble_matrix_rhs(g, data)
p_tpfa = scipy.sparse.linalg.spsolve(A, b_flow + fc)
# Two-point flux approximation - compute flux
flux = matrix_dictionary["flux"]
bound_flux = matrix_dictionary["bound_flux"]
div = pp.fvutils.scalar_divergence(g)
flux_tpfa = flux * p_tpfa + bound_flux * p_bound
######## MPFA
# Multi-point flux approximation - compute pressure
mpfa_solver = pp.Mpfa(data_key)
mpfa_solver.discretize(g, data)
A, b_flow = mpfa_solver.assemble_matrix_rhs(g, data)
p_mpfa = scipy.sparse.linalg.spsolve(A, b_flow + fc)
# Multi-point flux approximation - compute flux
flux  = matrix_dictionary["flux"]
bound_flux = matrix_dictionary["bound_flux"]
flux_mpfa = flux * p_mpfa + bound_flux * p_bound

# Exact solution

p_diff_tpfa = p_tpfa - pc_ex  
flux_diff_tpfa = flux_tpfa - flux_ex

p_diff_mpfa = p_mpfa - pc_ex  
flux_diff_mpfa = flux_mpfa - flux_ex

p_err_tpfa = (np.sqrt(np.sum(g.cell_volumes * (p_diff_tpfa)**2))/np.sqrt(np.sum(g.cell_volumes * pc_ex**2)))
flux_err_tpfa = (np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff_tpfa**2))/np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
     
p_err_mpfa = (np.sqrt(np.sum(g.cell_volumes * (p_diff_mpfa)**2))/np.sqrt(np.sum(g.cell_volumes * pc_ex**2)))
flux_err_mpfa = (np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff_mpfa**2))/np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
        
print('pressure error TPFA =', p_err_tpfa)
print('flux error TPFA =', flux_err_tpfa)

print('pressure error MPFA =', p_err_mpfa)
print('flux error MPFA =', flux_err_mpfa)

F_n = flux_mpfa * g.face_normals
pp.plot_grid(g, vector_value=F_n, figsize=(15, 12))



print('done')


/home/haakon/anaconda3/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py:753: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.0, top=0.0
  'bottom=%s, top=%s') % (bottom, top))


<Figure size 800x800 with 2 Axes>

pressure error TPFA = 0.08987306778412095
flux error TPFA = 0.1027300650313017
pressure error MPFA = 0.0395465747185387
flux error MPFA = 0.04301106780442411


/home/haakon/anaconda3/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py:753: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.0, top=0.0
  'bottom=%s, top=%s') % (bottom, top))


<Figure size 1500x1200 with 1 Axes>

done
